#### Bring in Required Libraries

In [72]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from icecream import ic
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [73]:
# ODBC Connection for FileMaker
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor_fm = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 10.222 sec


In [74]:
# SQLAlchemy connection for SQL Server
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn_sql = engine.connect()

In [75]:
# ODBC Connection for AS400
CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""
dbcnxn = pyodbc.connect(CONNAS400)
cursor_as400 = dbcnxn.cursor()

In [93]:
sql_spares = """
    SELECT PROD.FPSPRMAST1.SPH_PART,
        STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
        STRIP(PROD.FPSPRMAST1.SPH_DESC1),
        STRIP(PROD.FPSPRMAST1.SPH_DESC2),
        STRIP(PROD.FPSPRMAST1.SPH_MFG),
        STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
        STRIP(PROD.FPSPRMAST2.SPD_CABINT),
        STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
        PROD.FPSPRMAST2.SPD_QOHCUR,
        PROD.FPSPRMAST1.SPH_CURSTD,
        STRIP(PROD.FPSPRMAST2.SPD_REODTE),
        STRIP(PROD.FPSPRMAST2.SPD_USECC),
        STRIP(PROD.FPSPRMAST2.SPD_PURCC),
        STRIP(PROD.FPSPRMAST2.SPD_QREORD)
    FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
    WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))
"""

In [94]:
sql_parts = """
    SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
        "Screw Part Number" AS Screw, "Band Feed from Band data",
        "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
        "CameraInspectionRequired", "ScrDrvChk"
    FROM tbl8Tridon
"""

In [108]:
sql_bands = """
    SELECT "Band Part Number", "Feed Length","CutoutA Tool Number","CutoutB Tool Number","CutoutC Tool Number",
        "Dim A", "Dim B","Dim C", "Process", "Description"
    FROM BANDS
"""

### Execute Query on AS400

In [96]:
start = timer()
cursor_as400.execute(sql_spares)
result_spares = cursor_as400.fetchall()
print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

AS400 Connect/Query Time = 2.117 sec


#### Execute Query on SQL Server

In [97]:
# Clamp Data
start = timer()
cursor_fm.execute(sql_parts)
result_clamp = cursor_fm.fetchall()
print("FileMaker Clamp Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Clamp Query Time = 3.078 sec


In [109]:
# Band Data
start = timer()
cursor_fm.execute(sql_bands)
result_band = cursor_fm.fetchall()
print("FileMaker Band Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Band Query Time = 0.921 sec


#### Build Dataframe for Spare Parts

In [100]:
data_type_dict = {'StandardCost' : float, 'OnHand' : int, 'PartNum' : str, 'ReOrderPt' : int}
df_spares = pd.DataFrame.from_records(result_spares)
df_spares.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand', 'StandardCost','ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
df_spares = df_spares.dropna()
df_spares = df_spares.astype(data_type_dict)
df_spares = df_spares.convert_dtypes()
df_spares.head

<bound method NDFrame.head of        PartNum   EngPartNum                     Desc1  \
0         1056    091001056           PLATE WEAR SIDE   
1         1107    091001107           PLATE SIDE WEAR   
2         1114    091001114     PLATE SIDE WEAR FRONT   
3         1115    091001115           PLATE WEAR SIDE   
4         2001      4-40 H3  SP PT GUN TAP PLUG STYLE   
...        ...          ...                       ...   
22522  1041029   104-10-029              1ST FORM PAD   
22523  3031116  3031-10-016             CUTOFF INSERT   
22524  3031204  3031-20-004            CUT OFF INSERT   
22525  1786196    091786196    SCPW55-14-22  COUPLING   
22526   824091   091824091E  QS186EBQ8  SENSOR EMITER   

                         Desc2 Mfg        MfgPn Cabinet Drawer  OnHand  \
0                                                    C7     D1       4   
1                                                     '      '       0   
2                                                     '      ' 

#### Build Dataframe for Clamp Data

In [105]:
df_clamps = pd.DataFrame.from_records(result_clamp)
# Set Column Names
df_clamps.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk']
# Set Data Types
data_type_dict = {'PartNumber' : str, 'Band' : str,'Housing' : str,'Screw' : str, 'Feed' : float, 'DiaMax' : float,'DiaMin' : float,'HexSz' : str,'BandThickness' : float, 'BandWidth' : float,'CamInspect' : str,'ScrDrvChk' : str}

# Due Some Filtering and Data Cleansing
df_clamps = df_clamps[df_clamps.Feed != 'N/A']
df_clamps = df_clamps[1:]
df_clamps = df_clamps.astype(data_type_dict)
df_clamps['CamInspect'] = df_clamps['CamInspect'].str.upper()
df_clamps['ScrDrvChk'] = df_clamps['ScrDrvChk'].str.upper()
df_clamps['Feed'] = df_clamps['Feed'].round(3)
df_clamps['BandWidth'] = df_clamps['BandWidth'].round(3)
df_clamps['BandThickness'] = df_clamps['BandThickness'].round(3)
df_clamps.fillna({'DiaMax' : 0.0}, inplace=True)
df_clamps.fillna({'DiaMin' : 0.0}, inplace=True)
df_clamps['DiaMax'] = df_clamps['DiaMax'].round(3)
df_clamps['DiaMin'] = df_clamps['DiaMin'].round(3)
df_clamps = df_clamps.dropna()
df_clamps = df_clamps.convert_dtypes()
df_clamps.sample(20)



,PartNumber,Band,Housing,Screw,Feed,DiaMax,DiaMin,HexSz,BandThickness,BandWidth,CamInspect,ScrDrvChk
10106,600350200052,78212,7010900,2542003,7.94,0.0,0.0,None,0.028,0.62,NO,NO
5576,106970048175,2852048,2367025,1278001MC,12.45,3.58,3.5,"5/16""",0.025,0.5,NO,NO
4397,670040032055,2345032,2351045,1318001,8.8,0.0,2.32,"5/16""",0.022,0.5,NO,NO
5001,370140016052,2745016,2360013,1278001MC,6.05,1.4,1.28,"5/16""",0.025,0.5,NO,NO
4777,632006070,2713006,2703021,1073003,3.736,0.0,0.0,"5/16""",0.025,0.5,NO,NO
6083,644P52222,2943052,2360007,1250010MC,12.818,0.0,0.0,"5/16""",0.022,0.5,NO,NO
2775,335006222,1266006,1097002,1108010MC,4.14,0.0,0.87,"1/4""",0.023,0.312,NO,NO
1494,M233018102,2314018,1019030,1073003,6.46,0.0,0.0,"5/16""",0.022,0.562,NO,NO
4760,612080102,2366080,2360013,1213002,18.25,0.0,0.0,"5/16""",0.022,0.5,YES,NO
818,387006006,1098006,2161001,2166003,4.266,0.0,0.84,"1/4""",0.025,0.312,NO,NO


#### Build Dataframe for Band Data

In [171]:
valid_process = ['IN-LINE SINGLE NOTCH DIE', 'IN-LINE DOUBLE NOTCH DIE', 'IN-LINE BAND STAMPING', 'IN-LINE 105 NOTCH DIE', 'CUT & CURL']
df_bands = pd.DataFrame.from_records(result_band)
df_bands = df_bands[1:]
df_bands.columns = ['Band', 'FeedLength', 'CutOutA', 'CutOutB', 'CutOutC', 'DimA', 'DimB', 'DimC', 'Process', 'Description']
data_type_dict = {'Band' : str, 'FeedLength' : float,'CutOutA' : str,'CutOutB' : str,'CutOutC' : str,'DimA' : float,'DimB' : float,'DimC' : float,'Process' : str, 'Description' : str}
df_bands['Band'] = df_bands['Band'].str.upper()
df_bands['Process'] = df_bands['Process'].str.upper()
df_bands = df_bands[df_bands['Process'].isin(valid_process)]
df_bands = df_bands[df_bands['Band'].str.contains('OBS|X|OLD|PSR|CH|NO ACTIVE') == False]
df_bands['FeedLength'] = df_bands['FeedLength'].round(3)
df_bands.fillna({'CutOutA' : 'NON-MULTI'}, inplace=True)
df_bands.fillna({'DimA' : 0.000}, inplace=True)
df_bands.fillna({'DimB' : 0.000}, inplace=True)
df_bands.fillna({'DimC' : 0.000}, inplace=True)
df_bands.drop_duplicates(subset='Band', keep='first', inplace=True)
df_bands = df_bands.dropna(subset='Band')
df_bands = df_bands.astype(data_type_dict)
#df_bands.query('BandLength < 10.000', inplace=True)
df_bands = df_bands.convert_dtypes()
# df_bands.sample(20)
# df_bands.head(20)
df_bands.dtypes

Band           string[python]
FeedLength            Float64
CutOutA        string[python]
CutOutB        string[python]
CutOutC        string[python]
DimA                  Float64
DimB                  Float64
DimC                  Float64
Process        string[python]
Description    string[python]
dtype: object

#### Load Spare Parts to SQL Server Table

In [165]:
data_type_dict = {'StandardCost' : sqlalchemy.types.FLOAT, 'OnHand' : sqlalchemy.types.INT,'PartNum' : sqlalchemy.types.VARCHAR(255),'ReOrderPt' : sqlalchemy.types.INT, 'EngPartNum' : sqlalchemy.types.VARCHAR(255), 'Desc1' : sqlalchemy.types.VARCHAR(255),'Desc2' : sqlalchemy.types.VARCHAR(255),'Mfg' : sqlalchemy.types.VARCHAR(255),'MfgPn' : sqlalchemy.types.VARCHAR(255), 'Cabinet' : sqlalchemy.types.VARCHAR(255),'Drawer' : sqlalchemy.types.VARCHAR(255),'ReOrderDate' : sqlalchemy.types.VARCHAR(255),'DeptUse' : sqlalchemy.types.VARCHAR(255),'DeptPurch' : sqlalchemy.types.VARCHAR(255)}

df_spares.to_sql('tblSpares', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

28

#### Load Clamp to SQL Server Table

In [99]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR(255), 'Band' : sqlalchemy.types.VARCHAR(255),'Housing' : sqlalchemy.types.VARCHAR(255),'Screw' : sqlalchemy.types.VARCHAR(255), 'Feed' : sqlalchemy.types.Float, 'DiaMax' : sqlalchemy.types.Float,'DiaMin' : sqlalchemy.types.Float,'HexSz' : sqlalchemy.types.VARCHAR(255),'BandThickness' : sqlalchemy.types.Float, 'BandWidth' : sqlalchemy.types.Float,'CamInspect' : sqlalchemy.types.VARCHAR(255),'ScrDrvChk' : sqlalchemy.types.VARCHAR(255)}

df_clamps.to_sql('parts_clamps', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

32

#### Load Band Data to SQL Server Table

In [172]:
data_type_dict = {'Band' : sqlalchemy.types.VARCHAR(255),'FeedLength' : sqlalchemy.types.FLOAT, 'CutOutA' : sqlalchemy.types.VARCHAR(255), 'CutOutB' : sqlalchemy.types.VARCHAR(255), 'CutOutC' : sqlalchemy.types.VARCHAR(255),'DimA' : sqlalchemy.types.FLOAT,'DimB' : sqlalchemy.types.FLOAT,'DimC' : sqlalchemy.types.FLOAT, 'Process' : sqlalchemy.types.VARCHAR(255), 'Description' : sqlalchemy.types.VARCHAR(255)}

df_bands.to_sql('parts_bands', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

16